# In this script I am trying to calculate the volume of the predicted shapes, with the help of code of the SHAPR-Framework. 

https://github.com/marrlab/SHAPR_torch/blob/master/docs/pre-post-processing/Extract%20Features%20from%203D%20Shapes.ipynb

The code from shapr, first normalizes the prediction data (64,64,64) which puts every pixel in relation to its highest pixel value. THe threshold_otsu method then finds an optimal threshold to the data. Every pixel below this threshold is put to zero and everything above it to one. The code then counts those ones which can then be multiplied with the metric to pixel ratio to get the volume in femtoliters respectively in cubic micrometers. 


In [ ]:
#import dependencies
import os
import numpy as np
from skimage.io import imread
from skimage.filters import threshold_otsu
import csv

In [ ]:
# set the path to the 2D segmentations ("mask") and the folder where the features should be saved
test_path = "Results"
out_path = "Data_Analysis"

### Trying it first on a single cell prediction

In [ ]:
#normalizes the data (putting every pixel in relation to the maximum pixel value of the data ndarray)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html
def norm_thres(data): 
    maxd = np.max(data)
    data = data / maxd
    data = np.nan_to_num(data)
    if np.max(data) > 0:
        thresh = threshold_otsu(data)
        binary = data > thresh
    else: 
        binary = data
    return binary*1.0

In [ ]:
#read the data
import tifffile
data_in = imread("Results/cell_frame004148_x0126_y0201_red.tif")
obj_auto_pred = np.nan_to_num(norm_thres(data_in))
auto_pred_vol = np.sum(obj_auto_pred)
tifffile.imwrite("Created_Images/output1.tif", obj_auto_pred*255 ) #*255 to visualize the cell after



print(auto_pred_vol) # counts all the ones in the 64,64,64 array



### Multipy with metric to pixel ratio

In [ ]:
from PIL import Image
desired_width, desired_height = 64, 64
desired_aspect_ratio = desired_width / desired_height

original_image = Image.open("Images_for_prediction/cell_frame004148_x0126_y0201_red.jpg")

current_width, current_height = original_image.size
current_aspect_ratio = current_width / current_height

if current_aspect_ratio > desired_aspect_ratio:
    # The original image is wider than the desired aspect ratio, so the width will be the limiting factor.
    scale_factor = desired_width / current_width
else:
    # The original image is taller than the desired aspect ratio, so the height will be the limiting factor.
    scale_factor = desired_height / current_height


metric_to_pixel = (75.5/1000) #75.5nm/px ; /1000 => micrometer/px; 

metric_to_pixel = (metric_to_pixel / scale_factor )  ** 3 #new metric to pixel value because the input image was resized (without loosing aspect ratio); ^3 to get cubic-micrometer

volume = auto_pred_vol * metric_to_pixel # cubic micrometer
int(volume) #in femtoliters


### Save in an excel with all the files and applying the metric to pixel value with the scaling factor from altering the images

In [ ]:
import os
from PIL import Image
import numpy as np
desired_width, desired_height = 64, 64
desired_aspect_ratio = desired_width / desired_height

# Set the path to the folder containing the images
folder_path = "Images_for_prediction"  

scale_factors = []


# Loop over all the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Load the original image
        file_path = os.path.join(folder_path, filename)
        original_image = Image.open(file_path)

        current_width, current_height = original_image.size
        current_aspect_ratio = current_width / current_height

        if current_aspect_ratio > desired_aspect_ratio:
            # The original image is wider than the desired aspect ratio, so the width will be the limiting factor.
            scale_factor = desired_width / current_width
            scale_factors.append(scale_factor)
        else:
            # The original image is taller than the desired aspect ratio, so the height will be the limiting factor.
            scale_factor = desired_height / current_height
            scale_factors.append(scale_factor)

In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.filters import threshold_otsu
import openpyxl

#normalizes the data (putting every pixel in relation to the maximum pixel value of the data ndarray)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html

def norm_thres(data):
    maxd = np.max(data)
    data = data / maxd
    data = np.nan_to_num(data)
    if np.max(data) > 0:
        thresh = threshold_otsu(data)
        binary = data > thresh
    else:
        binary = data
    return binary.astype(float)

folder_path = "Results"
out_path = "Data_Analysis"

# Create a new workbook and get the active sheet
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.column_dimensions["A"].width = 100

sheet["A1"] = "filename"
sheet["B1"] = "image_scale_factor"
sheet["C1"] = "count"
sheet["D1"] = "metric_to_pixel"
sheet["E1"] = "volume"


for index, filename in enumerate(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)

    #read the data
    data_in = imread(file_path)
    obj_auto_pred = np.nan_to_num(norm_thres(data_in)) #normalizes the data and puts every pixel above certain threshold to 1
    auto_pred_vol = np.sum(obj_auto_pred) #counts the 1s
    
    #calculate in cubicmicrometers
    metric_to_pixel = (75.5/1000) #75.5nm/px ; /1000 => micrometer/px; 
    scaling_factor = scale_factors[index] 
    metric_to_pixel = (metric_to_pixel / scaling_factor )  ** 3 #new metric to pixel value because the input image was resized (without loosing aspect ratio); ^3 to get cubic-micrometer


    volume = auto_pred_vol * metric_to_pixel # cubic micrometer
    # Write data rows in excel worksheet
    row = [filename, scaling_factor, auto_pred_vol, metric_to_pixel, volume]
    sheet.append(row)

output_file = os.path.join(out_path, 'Volume_Calculation_bySHAPR.xlsx')
workbook.save(output_file)







